[Reference](https://medium.com/vacatronics/6-ways-to-use-metaclasses-in-python-8c033dbd225d)

# 1. Class Registration

In [1]:
class Animal:
    def sound(self):
        print(f"Hey, I'm {self.name}")


class Dog(Animal):
    name = "Rex"


class Cat(Animal):
    name = "Kitty"

In [2]:
class AnimalFactory:
    animals = {}

    @classmethod
    def from_name(cls, name):
        animal_class = cls.animals.get(name.lower())
        if not animal_class:
            raise ValueError("Animal not found")

        return animal_class()

    @classmethod
    def register(cls, name, animal_class):
        cls.animals[name] = animal_class

In [3]:
class AnimalMeta(type):
    def __new__(cls, name, bases, namespace):
        new_cls = super().__new__(cls, name, bases, namespace)
        AnimalFactory.register(name.lower(), new_cls)
        return new_cls

In [4]:
class AnimalFactory:
    animals = {}

    @classmethod
    def from_name(cls, name):
        animal_class = cls.animals.get(name.lower())
        if not animal_class:
            raise ValueError("Animal not found")

        return animal_class()

    @classmethod
    def register(cls, name, animal_class):
        cls.animals[name] = animal_class


class AnimalMeta(type):
    def __new__(cls, name, bases, namespace):
        new_cls = super().__new__(cls, name, bases, namespace)
        AnimalFactory.register(name.lower(), new_cls)
        return new_cls


class Animal(metaclass=AnimalMeta):
    name = "animal"

    def sound(self):
        print(f"Hey, I'm {self.name}!")


class Dog(Animal):
    name = "Rex"


class Cat(Animal):
    name = "Kitty"


cat = AnimalFactory.from_name("cat")
cat.sound()
dog = AnimalFactory.from_name("dog")
dog.sound()

Hey, I'm Kitty!
Hey, I'm Rex!


# 2. Attributes Modification

In [5]:
class Field:
    name: str


class Model:
    x = Field()
    y = Field()
    z = Field()

In [6]:
class Field:
    name: str


class MetaModel(type):
    def __new__(cls, name, bases, namespace):
        fields = {}
        def mod_fields(items):
            for key, value in items:
                if isinstance(value, Field):
                    value.name = f"{name}.{key}"
                    fields[key] = value
        
        mod_fields(namespace.items())
        for base in bases:
            if hasattr(base, '_fields'):
                mod_fields(base._fields.items())
        
        namespace['_fields'] = fields
        return super().__new__(cls, name, bases, namespace)


class Model(metaclass=MetaModel):
    x = Field()
    y = Field()
    z = Field()


class SubModel(Model):
    w = Field()


sm = SubModel()
for f in sm._fields.values():
    print(f.name)

SubModel.w
SubModel.x
SubModel.y
SubModel.z


# 3. Final Class

In [7]:
class FinalMeta(type):
    def __new__(cls, name, bases, namespace):
        for base in bases:
            if type(base) is FinalMeta:
                raise RuntimeError("You cannot subclass a final class")
        return super().__new__(cls, name, bases, namespace)


class Person(metaclass=FinalMeta):
    def run(self):
        print("Run, Forrest, run!")


class OtherPerson(Person):
    def walk(self):
        print("Walking...")

RuntimeError: ignored

# 4. Singleton

In [8]:
class Singleton(type):
    _instance = None

    def __call__(cls, *args, **kwds):
        if not cls._instance:
            cls._instance = super().__call__(*args, **kwds)
        return cls._instance


class A(metaclass=Singleton):
    pass


a1 = A()
a2 = A()

print(a1 is a2)
print(a1.__class__, a2.__class__)

## Output:

True
<class '__main__.A'> <class '__main__.A'>


# 5. Validation

In [9]:
class Interface(type):

    def __new__(cls, name, bases, namespace):
        # This is very simple, but just to give an example
        if len(bases) == 0:
            return super().__new__(cls, name, bases, namespace)
        
        for key in bases[0].__dict__.keys():
            if key.startswith("_"):
                continue

            if key not in namespace or not callable(namespace[key]):
                raise TypeError(f"Class {name} should implement method {key}")

        return super().__new__(cls, name, bases, namespace)


class WriterInterface(metaclass=Interface):
    def open(self):
        pass

    def close(self):
        pass

    def write(self):
        pass


class FileWriter(WriterInterface):
    def open(self):
        pass

TypeError: ignored

# 6. Declarative form

In [12]:
root = Element()
child1 = Element()
child2 = Element()
child11 = Element()

child1.add(child11)
root.add(child1)
root.add(child2)

In [11]:
class Root:
    class Child1:
        class Child11:
            pass
    
    class Child2:
        pass